In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from matplotlib import pyplot as plt
from ydata_profiling import ProfileReport

from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

pd.options.display.max_columns = None

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Leemos el dataset, que está en formato CSV desde google drive.
Adicionalmente renombro a las columnas en un formato mas comodo de manejar

In [61]:
df = pd.read_csv(
    'https://drive.google.com/uc?export=download&id=1gq-wDn_dwz_5uHSEoMYmQtnNrmnfNiXS'
)
df.rename(columns={c: c.lower().replace(" ", "_") for c in df.columns}, inplace=True)

In [42]:
df.head()

,name,gender,eye_color,race,hair_color,height,publisher,skin_color,alignment,weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


In [45]:
df.skin_color.replace('-',np.nan)

0        NaN
1       blue
2        red
3        NaN
4        NaN
       ...  
729      NaN
730    white
731    green
732      NaN
733      NaN
Name: skin_color, Length: 734, dtype: object

In [46]:
df.skin_color.value_counts()

skin_color
-                 662
green              21
red                 9
blue                9
white               7
grey                5
silver              5
purple              3
gold                3
yellow              2
pink                2
red / black         1
black               1
orange / white      1
gray                1
blue-white          1
orange              1
Name: count, dtype: int64

In [49]:
df.skin_color.replace('-','no color')

0      no color
1          blue
2           red
3      no color
4      no color
         ...   
729    no color
730       white
731       green
732    no color
733    no color
Name: skin_color, Length: 734, dtype: object

In [17]:
df_orig = df

# Missings (Trabajando con valores faltantes)

Veamos que variables contienen nulos

In [4]:
df.isnull().sum().to_frame('Cantidad nulls')

,Cantidad nulls
name,0
gender,0
eye_color,0
race,0
hair_color,0
height,0
publisher,15
skin_color,0
alignment,0
weight,2


In [6]:
# Aca vemos como pasar el nan al texto "nan" y por lo tanto es un "nuevo" color
df['publisher'].astype(str).unique()

array(['Marvel Comics', 'Dark Horse Comics', 'DC Comics', 'NBC - Heroes',
       'Wildstorm', 'Image Comics', 'nan', 'Icon Comics', 'SyFy',
       'Hanna-Barbera', 'George Lucas', 'Team Epic TV', 'South Park',
       'HarperCollins', 'ABC Studios', 'Universal Studios', 'Star Trek',
       'IDW Publishing', 'Shueisha', 'Sony Pictures', 'J. K. Rowling',
       'Titan Books', 'Rebellion', 'Microsoft', 'J. R. R. Tolkien'],
      dtype=object)

In [50]:
df['publisher'].value_counts()

publisher
Marvel Comics        388
DC Comics            215
NBC - Heroes          19
Dark Horse Comics     18
George Lucas          14
Image Comics          14
Star Trek              6
HarperCollins          6
SyFy                   5
Team Epic TV           5
Icon Comics            4
ABC Studios            4
IDW Publishing         4
Shueisha               4
Wildstorm              3
Sony Pictures          2
Hanna-Barbera          1
South Park             1
Universal Studios      1
J. K. Rowling          1
Titan Books            1
Rebellion              1
Microsoft              1
J. R. R. Tolkien       1
Name: count, dtype: int64

### Opcion 0: Tratarla como una "categoria" o valor más
Vimos un ejemplo con el One Hot Encoder

In [8]:
df.publisher.astype(str).value_counts().to_frame('cantidad')

,cantidad
publisher,
Marvel Comics,388
DC Comics,215
NBC - Heroes,19
Dark Horse Comics,18
nan,15
George Lucas,14
Image Comics,14
HarperCollins,6
Star Trek,6


In [11]:
df['publisher'].fillna('sin datos',inplace=True)
df['publisher'] = df['publisher'].fillna('sin datos')

In [12]:
df.isnull().sum().to_frame('Cantidad nulls')

,Cantidad nulls
name,0
gender,0
eye_color,0
race,0
hair_color,0
height,0
publisher,0
skin_color,0
alignment,0
weight,2


### Opcion 1: remover los nulos del dataset

In [51]:
(df.isnull().mean() * 100).to_frame('porcentaje nulls')

,porcentaje nulls
name,0.000000
gender,0.000000
eye_color,0.000000
race,0.000000
hair_color,0.000000
height,0.000000
publisher,2.043597
skin_color,0.000000
alignment,0.000000
weight,0.272480


Veamos algunos registros de dichas variables accediendo con [.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)

In [30]:
df_orig[df_orig.isnull().any(axis=1)][['publisher','weight']]

(15, 2)

Tenemos dos tipos de variables a tratar, numéricas y categóricas. <br>
Veamos algunas soluciones generales

In [29]:
# eliminar filas con nulos

less_rows = df.dropna(axis=0)
len(less_rows)

719

In [32]:
df.shape

(734, 10)

Eliminamos 15 instancias

In [33]:
# eliminar filas con alto porcentaje de nulos

NULL_REMOVE_PERCENT = 0.30
df[df.isnull().mean(axis=1) < NULL_REMOVE_PERCENT]

,name,gender,eye_color,race,hair_color,height,publisher,skin_color,alignment,weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0
...,...,...,...,...,...,...,...,...,...,...
729,Yellowjacket II,Female,blue,Human,Strawberry Blond,165.0,Marvel Comics,-,good,52.0
730,Ymir,Male,white,Frost Giant,No Hair,304.8,Marvel Comics,white,good,-99.0
731,Yoda,Male,brown,Yoda's species,White,66.0,George Lucas,green,good,17.0
732,Zatanna,Female,blue,Human,Black,170.0,DC Comics,-,good,57.0


In [34]:
# eliminar columnas con nulos

less_cols = df.dropna(axis=1)
less_cols.shape

(734, 8)

In [39]:
# eliminar columnas con alto porcentaje de nulos

NULL_REMOVE_PERCENT = 0.01
cols = df.isna().mean()
cols = cols[cols < NULL_REMOVE_PERCENT]
df[cols.index]

,name,gender,eye_color,race,hair_color,height,skin_color,alignment,weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,-,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,-,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,-,bad,-99.0
...,...,...,...,...,...,...,...,...,...
729,Yellowjacket II,Female,blue,Human,Strawberry Blond,165.0,-,good,52.0
730,Ymir,Male,white,Frost Giant,No Hair,304.8,white,good,-99.0
731,Yoda,Male,brown,Yoda's species,White,66.0,green,good,17.0
732,Zatanna,Female,blue,Human,Black,170.0,-,good,57.0


ELiminamos una columna

## Opcion 2: completar usando info de esa columna (Univariadas)

#### Categóricas

Como vimos, los encoders solucionan el problema de nulos ya que imputan con la misma lógica que para los demás valores de la variable

#### Numéricas

Completar con la mediana, promedio, moda o constante

In [40]:
def show_strategies(df, name_col, k=-99):
    '''Devuelve el valor de imputacion de las tres estrategias para esa columna'''

    _df = df[[name_col]].copy()
    s = df[name_col]

    _df['median'] = s.fillna(s.median())
    _df['mean'] = s.fillna(s.mean())
    _df['mode'] = s.fillna(s.mode()[0])
    _df['contant'] = k

    # vemos los valores con los que completa en cada caso
    return _df[s.isna()]


show_strategies(df, 'weight')

,weight,median,mean,mode,contant
286,NaN,62.0,43.855191,-99.0,-99
389,NaN,62.0,43.855191,-99.0,-99


> Si implementamos el mismo ejemplo con sklearn, aparece el concepto de *imputer*

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html

lo salteo

In [54]:
def compare_imputers(df, name_col, k):
    '''Devuelve el valor de imputacion de las estrategias para esa columna'''

    median_imputer = SimpleImputer(strategy='median', fill_value=k)
    mean_imputer = SimpleImputer(strategy='mean', fill_value=k)
    mode_imputer = SimpleImputer(strategy='most_frequent', fill_value=k)
    constant_imputer = SimpleImputer(strategy='constant', fill_value=-99)

    _df = df.copy()
    _df['median'] = median_imputer.fit_transform(df[[name_col]])
    _df['mean'] = mean_imputer.fit_transform(df[[name_col]])
    _df['mode'] = mode_imputer.fit_transform(df[[name_col]])
    _df['constant'] = constant_imputer.fit_transform(df[[name_col]])

    return _df[[name_col, 'median', 'mode', 'constant']]

In [53]:
df.height.fillna(df.height.mean())

0      203.0
1      191.0
2      185.0
3      203.0
4      -99.0
       ...  
729    165.0
730    304.8
731     66.0
732    170.0
733    185.0
Name: height, Length: 734, dtype: float64

In [55]:

_df = compare_imputers(df, 'weight', -99)
display(_df[_df['weight'].isna()].head(5))


_df = compare_imputers(df, 'height', -99)
display(_df[_df['height'].isna()].head(5))


,weight,median,mode,constant
286,NaN,62.0,-99.0,-99.0
389,NaN,62.0,-99.0,-99.0


,height,median,mode,constant


## Opcion 3: completar usando info de las demas columnas (Multivariada)

Usa información de todas las variables para la imputación.<br>

Veamos un ejemplo con KNN (lo verán en detalle en las próximas clases).

In [ ]:
def hashing_encoding(df, cols, data_percent=0.85, verbose=False):
    for i in cols:
        val_counts = df[i].value_counts(dropna=False)
        s = sum(val_counts.values)
        h = val_counts.values / s
        c_sum = np.cumsum(h)
        c_sum = pd.Series(c_sum)
        n = c_sum[c_sum > data_percent].index[0]
        if verbose:
            print("n hashing para ", i, ":", n)
        if n > 0:
            fh = FeatureHasher(n_features=n, input_type='string')
            hashed_features = fh.fit_transform(
                df[i].astype(str).values.reshape(-1, 1)
            ).todense()
            df = df.join(pd.DataFrame(hashed_features).add_prefix(i + '_'))

    return df.drop(columns=cols)


def knn_imputer(df):

    cat_cols = ['gender', 'eye_color', 'race', 'hair_color', 'publisher', 'skin_color']

    # Aplicamos hashing para las categoricas
    df = hashing_encoding(df, cat_cols)

    # Eliminamos name y alignment para imputar
    df = df.drop(columns=['name', 'alignment'])

    # definimos un n arbitrario
    imputer = KNNImputer(n_neighbors=2, weights="uniform")
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    return df


knn_imputation = knn_imputer(df).add_suffix('_knn')
display(
    df[['name', 'weight', 'height']]
    .join(knn_imputation[['weight_knn', 'weight_knn']])[
        (df.weight.isna() | df.height.isna())
    ]
    .head(5)
)

IterativeImputer  
https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html

In [56]:
df[df.weight.isna()]

,name,gender,eye_color,race,hair_color,height,publisher,skin_color,alignment,weight
286,Godzilla,-,-,Kaiju,-,108.0,NaN,grey,bad,NaN
389,King Kong,Male,yellow,Animal,Black,30.5,NaN,-,good,NaN


In [63]:
df['weight_was_null'] = False
df.loc[df.weight.isna(),'weight_was_null'] = True

In [67]:
df['weight'].fillna(-99,inplace=True)

In [68]:
df[df['weight_was_null'] == True]

,name,gender,eye_color,race,hair_color,height,publisher,skin_color,alignment,weight,weight_was_null
286,Godzilla,-,-,Kaiju,-,108.0,NaN,grey,bad,-99.0,True
389,King Kong,Male,yellow,Animal,Black,30.5,NaN,-,good,-99.0,True


***Comentario Final***:
- A veces va a ayudar a los modelos que le digamos explicitamente que ese valor fue "calculado", eso
le puede permitir al modelo elegir si darle un poco de menos importancia (relativa a las originales) si fue calculado.